In [ ]:
import numpy as np
import torch
from tibo1.Lab3.utils import get_device
from tibo1.Lab3.model import UNet
import pickle
import gzip
from tqdm import tqdm

### 1. Submission Generation
#### 1a. Load Preprocessed test data

In [ ]:
# Load Preprocessed test data
test = np.load('./tibo1/Lab3/out/preprocessed/test.npy', allow_pickle=True)
print(test.shape)

In [7]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        return pickle.load(f)

# Load test metadata
test_data = load_zipped_pickle('./tibo1/Lab3/data/test.pkl')

# Generate test frame names
test_names = []
for patient in tqdm(test_data, desc="Generating Test Names"):
    video_name = patient['name']
    n_frames = patient['video'].shape[-1]
    test_names.extend([f"{video_name}_{frame}" for frame in range(n_frames)])

print(f"Generated {len(test_names)} test frame names.")
assert len(test_names) == sum([p['video'].shape[-1] for p in test_data]), "Mismatch between test frames and names."


Generating Test Names: 100%|██████████| 20/20 [00:00<00:00, 51181.26it/s]

Generated 1507 test frame names.


In [9]:
device = get_device()
print(f"Running on device: {device}")

# Load model
model = UNet(n_channels=1, n_classes=1, bilinear=False)
model_path = './tibo1/Lab3/out/checkpoints/best_model_epoch_1.pth'
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()
model.to(device)
print(f"Loaded model from {model_path}.")


Running on device: cuda
Loaded model from ./tibo1/Lab3/out/checkpoints/best_model_epoch_1.pth.


#### Make predictions

In [11]:
predictions = []

for patient in tqdm(test, desc="Predicting test frames"):
    video_name = patient['name']
    video_frames = patient['video']  # Extract the video frames from the dictionary

    for frame_idx in range(video_frames.shape[-1]):  # Iterate over each frame in the video
        # Extract and process the individual frame
        video_frame = video_frames[..., frame_idx]
        frame_tensor = torch.tensor(video_frame, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

        with torch.no_grad():
            # Predict mask for the frame
            pred_mask = model(frame_tensor)
            pred_mask = (pred_mask > 0.5).float().cpu().numpy().squeeze()

        # Append prediction with the correct name
        predictions.append((f"{video_name}_{frame_idx}", pred_mask))

print(f"Generated predictions for {len(predictions)} frames.")

Predicting test frames: 100%|██████████| 20/20 [00:42<00:00,  2.14s/it]

Generated predictions for 4480 frames.


#### Format Predictions in Correct Format

In [14]:
def get_sequences(arr):
    """
    Extracts sequences of 1s from a flattened array for submission format.
    """
    indices, lengths = [], []
    in_sequence = False
    start_idx = 0
    for i, val in enumerate(arr):
        if val == 1 and not in_sequence:
            start_idx = i
            in_sequence = True
        elif val == 0 and in_sequence:
            indices.append(start_idx)
            lengths.append(i - start_idx)
            in_sequence = False
    if in_sequence:
        indices.append(start_idx)
        lengths.append(len(arr) - start_idx)
    return indices, lengths


ids = []
values = []

for name, mask in tqdm(predictions, desc="Formatting predictions"):
    flat_mask = mask.flatten()
    indices, lengths = get_sequences(flat_mask)
    for start, length in zip(indices, lengths):
        ids.append(name)
        values.append([start, length])


Formatting predictions:  19%|█▉        | 868/4480 [02:09<08:59,  6.69it/s]


KeyboardInterrupt: 

In [15]:
print(f"Total predictions: {len(predictions)}")
for i in range(3):  # Print the first three predictions
    print(predictions[i])

Total predictions: 4480
('E9AHVWGBUF_0', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))
('E9AHVWGBUF_1', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))
('E9AHVWGBUF_2', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))


In [16]:
print(pred_mask.max(), pred_mask.min())

0.0 0.0


#### Save Predictions

In [ ]:
submission_df = pd.DataFrame({
    "id": ids,
    "value": [str(value) for value in values]
})

submission_file = "./tibo1/Lab3/out/1_submission_best_model_fine_tuned_epoch1.csv"
submission_df.to_csv(submission_file, index=False)
print(f"Submission saved to {submission_file}")

In [17]:
import cv2

# Save some frames and their masks for debugging
for i in range(3):  # Save first three frames
    cv2.imwrite(f"./tibo1/Lab3/debug/frame_{i}.png", test[i] * 255)
    cv2.imwrite(f"./tibo1/Lab3/debug/pred_mask_{i}.png", predictions[i][1] * 255)


TypeError: unsupported operand type(s) for *: 'dict' and 'int'

In [18]:
for name, mask in predictions[:3]:  # Debug first three predictions
    flat_mask = mask.flatten()
    indices, lengths = get_sequences(flat_mask)
    print(f"Name: {name}")
    print(f"Mask non-zero count: {np.count_nonzero(flat_mask)}")
    print(f"Sequences (indices): {indices}")
    print(f"Sequences (lengths): {lengths}")


Name: E9AHVWGBUF_0
Mask non-zero count: 0
Sequences (indices): []
Sequences (lengths): []
Name: E9AHVWGBUF_1
Mask non-zero count: 0
Sequences (indices): []
Sequences (lengths): []
Name: E9AHVWGBUF_2
Mask non-zero count: 0
Sequences (indices): []
Sequences (lengths): []


In [12]:
# Load a training frame for debugging
expert_X_train = np.load('./tibo1/Lab3/out/preprocessed/expert_X_train.npy')

# Use a sample from the expert training data
train_sample = expert_X_train[0]  # First preprocessed training image
train_tensor = torch.tensor(train_sample, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

# Predict the mask
with torch.no_grad():
    train_pred_mask = model(train_tensor)
    print(f"Train Sample Prediction - Max: {train_pred_mask.max()}, Min: {train_pred_mask.min()}, Mean: {train_pred_mask.mean()}")

Train Sample Prediction - Max: 0.0, Min: 0.0, Mean: 0.0
